In [23]:
import os
import numpy
from PIL import Image
import matplotlib.pyplot as plt

In [24]:
data_matrix = numpy.ndarray(shape=(400, 10304), dtype=numpy.dtype('u1'))  # 8-bit unsigned int
label_vector = numpy.empty(shape=(400, 1), dtype=numpy.dtype('u1'))

i = 0
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        image = Image.open(os.path.join(dirname, filename))
        numpy_image = numpy.asarray(image).flatten()
        data_matrix[i] = numpy_image.copy()
        label_vector[i] = int(dirname[dirname.rindex('\\') + 2:])
        i += 1

print(data_matrix)
print(data_matrix.shape)

[[ 48  49  45 ...  47  46  46]
 [ 34  34  33 ...  37  40  33]
 [ 60  60  62 ...  32  34  34]
 ...
 [112 109 116 ...  93  88  92]
 [111 114 112 ...  88  86  92]
 [110 112 113 ...  92  87  90]]
(400, 10304)


In [25]:
train_data=numpy.empty((280,10304))
y_train= numpy.empty((280,1))
test_data= numpy.empty((120,10304))
y_test= numpy.empty((120,1))

x=0
y=0
for i in range(40):
    for j in range(7):
        train_data[x][:]=data_matrix[(i*10)+j][:]
        y_train[x]=label_vector[(i*10)+j][0]
        x=x+1

    for k in range(3):
        test_data[y]=data_matrix[(i*10)+7+k][:]
        y_test[y]=label_vector[(i*10)+7+k][0]
        y=y+1

In [26]:
# PCA
def PCA(D, alpha):
    mean = numpy.mean(D, axis=0).reshape(1, train_data.shape[1])
    centered_data = numpy.subtract(D, mean)
    cov_matrix = numpy.cov(centered_data, rowvar=False)
    eigen_values, eigen_vectors = numpy.linalg.eigh(cov_matrix)

    #sort the eigenvalues and eigenvectors in descending order
    sorted_index = numpy.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:, sorted_index]

    sum_eigenvalues = sorted_eigenvalue.sum()
    sum_reduced = sorted_eigenvalue[0]

    components = 1
    while (sum_reduced / sum_eigenvalues) < alpha:
        sum_reduced += sorted_eigenvalue[components]
        components += 1

    reduced_basis = sorted_eigenvectors[:, 0:components]

    return reduced_basis

In [27]:
U = PCA(train_data, 0.9)
# u1    u2  ... ur
print(U.shape)

(10304, 89)


In [28]:
X_train = numpy.matmul(train_data, U)
X_test = numpy.matmul(test_data, U)

print(X_train.shape)
print(X_test.shape)

(280, 89)
(120, 89)


In [32]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train, y_train)

# Predict on dataset which model has not seen before
y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.95


C:\ProgramData\Anaconda3\envs\FaceRecognition\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
